<a href="https://colab.research.google.com/github/dasalgadob/handson-machine-learning/blob/master/3Imageretrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import stack
#from keras.datasets import mnist
from keras.datasets import mnist
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import os

## Embedding visualization

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
print(mnist_data[::1])

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f858f28>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f88b240>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f88b390>)


In [0]:
no_embedding_data = 1000
embedding_variable = tf.Variable(stack( mnist_data.test.images[:no_embedding_data], axis=0), trainable= False)

### MNIST test data to metadata file for visualization

In [0]:
metadata_path = '/metadata.tsv'
work_dir = ''

with open(metadata_path, 'w') as metadata_file:
  for i in range(no_embedding_data):
    metadata_file.write('{}\n'.format( np.nonzero(mnist_data.test.labels[::1])[1:][0][i]))
    
    
projector_config =projector.ProjectorConfig()
embedding_projection = projector_config.embeddings.add()
embedding_projection.tensor_name = embedding_variable.name
embedding_projection.metadata_path = metadata_path
embedding_projection.sprite.image_path = os.path.join( work_dir + 'mnist_10k_sprite.png')
embedding_projection.sprite.single_image_dim.extend([28,28])

In [0]:
#projector.visualize_embeddings(train_summary_writer, projector_config)
#tf.train.Saver().save(session, '/tmp/train/model.ckpt', global_step=1)